# Survey Pre-Processing for Cycling in Prague Study

## Introduction

This Jupyter Notebook demonstrates the pre-processing steps for a survey dataset on cycling in Prague. It's designed to help researchers and data analysts clean, transform, and prepare survey data for further analysis. This script is particularly useful for those working with survey data in urban planning, transportation studies, or social sciences.

### What You'll Learn

- How to import and handle various data types in Python
- Techniques for cleaning and standardizing survey responses
- Methods for handling geospatial data
- Approaches to categorizing and encoding survey responses
- Best practices for identifying and flagging potential data quality issues


### Libraries Used

We'll be using the following Python libraries:

- pandas: For data manipulation and analysis
- numpy: For numerical operations
- datetime: For handling date and time data
- json: For JSON data processing (if needed)
- geopandas: For geospatial data operations
- shapely: For geometric operations
- fuzzywuzzy: For string matching and comparison

Let's get started by importing these libraries and loading our data!

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime
import json
import geopandas as gpd
from shapely.geometry import Point
from fuzzywuzzy import process

Load the raw data from Qualtrix. 

pd.read_excel does two things. One it creates a pandas dataframe. This is a data structure like xlsx but for python.

In [ ]:
# File path
file_path = r"V:\Prague_Biking\Data\Survey Data\Cycling in Prague_October 9, 2024_14.08 - Copy.xlsx"
# Read the Excel file
df = pd.read_excel(file_path)

# Connecting to Geo-Areas 
Since our survey respondents don't know what a Geo-Area is, they have given us their work and home location interms of prague neighbourhoods. Many have misspellings and typos. 

Rossanne made a list of Geo-Areas corresponding to neighbourhoods. This is a slightly expanded version of that, where we take neighbourhoods which are close but not exactly in a geo-area. 

The algorithm is nearest geo-area gets a neighbourhood if within a 5km radius. 

We store the list of neighbourhoods and the corresponding Geo-Areas as a dictionary in python.

In [ ]:
# Define neighborhood dictionary
neighborhood_dict = {
    'Geo 1': ['Letná', 'Vinohrady', 'Nové Město', 'Josefov', 'Staré Město'],
    'Geo 2': ['Břevnov', 'Petřiny', 'Střešovice', 'Holešovice', 'Bořislavka', 'Dejvice'],
    'Geo 3': ['Podolí', 'Strahov', 'Smíchov', 'Nusle', 'Pankrác'],
    'Geo 4': ['Žižkov', 'Karlín', 'Hloubětín', 'Střížkov', 'Libeň', 'Vysočany', 'Kobylisy'],
    'Geo 5': ['Suchdol']
}

# Flatten the dictionary for easier fuzzy matching
all_neighborhoods = [item for sublist in neighborhood_dict.values() for item in sublist]

Matching neighborhoods to geo-areas

In our survey, respondents provided their home and work locations as Prague neighborhoods.
However, we want to group these neighborhoods into larger geo-areas for our analysis.

We're going to create a function that does two important things:
1. It corrects small spelling mistakes or inconsistencies in the neighborhood names.
2. It assigns each neighborhood to its corresponding geo-area.

This function will use a technique called "fuzzy matching". This is helpful because:
- It can handle minor spelling errors (e.g., "Vinhorady" instead of "Vinohrady")
- It can deal with missing diacritical marks (e.g., "Zizkov" instead of "Žižkov")
- It can match slight variations in naming (e.g., "Nove Mesto" vs "Nové Město")

By using this approach, we can clean up our data and group it into the geo-areas
we defined earlier, making our subsequent analysis more accurate and meaningful.

In [ ]:
def correct_neighborhood(neighborhood):
    if pd.isna(neighborhood):
        return np.nan, np.nan
    
    # Perform fuzzy matching
    match = process.extractOne(neighborhood, all_neighborhoods)
    if match[1] >= 80:  # 80% similarity threshold
        corrected = match[0]
        
        # Find the corresponding geo-area
        for geo, neighborhoods in neighborhood_dict.items():
            if corrected in neighborhoods:
                return corrected, geo
    
    return neighborhood, np.nan  # Return original if no match found

# Main Data Pre-processing Function

The `preprocess_data(df)` function is the core of our data cleaning and preparation process. It takes the raw survey data and transforms it into a format ready for analysis. This function performs several key operations:

1. Basic data cleaning (removing empty rows, handling missing data)
2. Date and time handling
3. Text response cleaning and standardization
4. Neighborhood correction and geo-area assignment
5. Identification of potential print surveys
6. Consistency checks for conditional questions
7. Encoding of multiple-choice questions
8. Age categorization
9. Survey duration calculation
10. Data quality flagging
11. Language identification
12. Likert scale processing

This comprehensive process ensures our data is clean, consistent, and properly formatted for in-depth analysis of cycling patterns in Prague.

In [1]:
import warnings
warnings.filterwarnings('ignore')

def preprocess_data(df):
    # Debugging: Print column names and data types
    print("Column names:", df.columns.tolist())
    print("\nData types:\n", df.dtypes)
    
    # Step 1: Basic Data Cleaning
    df = df.replace({'': np.nan, ' ': np.nan})  # Replace empty strings and spaces with NaN
    df = df.dropna(how='all')  # Drop rows that are all NaN
    world_shapefile_path = r"V:\Prague_Biking\Data\Maps\ne_110m_admin_0_countries.shp"
    df = process_ip_locations(df, world_shapefile_path)
    # Step 2: Handle dates
    date_columns = ['StartDate', 'EndDate', 'RecordedDate']
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

    # Step 3: Clean text responses
    text_columns = ['Q7', 'Q9', 'Q11', 'Q14_9_TEXT', 'Q17_9_TEXT', 'Q21_10_TEXT', 'Q22_10_TEXT', 'Q28_7_TEXT', 'Q29_6_TEXT', 'Q33_6_TEXT', 'Q34']
    for col in text_columns:
        if col in df.columns:
            df[col] = df[col].str.strip().str.lower()
    
    if 'Q7' in df.columns:
        df['Q7_corrected'], df['Q7a'] = zip(*df['Q7'].apply(correct_neighborhood))
    
    if 'Q11' in df.columns:
        df['Q11_corrected'], df['Q8a'] = zip(*df['Q11'].apply(correct_neighborhood))
    # Step 4: Identify potential print surveys
    df['potential_print_survey'] = df['RecordedDate'].dt.date == datetime(2024, 10, 8).date()

    # Step 5: Handle conditional questions
    conditional_pairs = [
        ('Q6', 'Q7'),  # If lives in Prague, neighborhood
        ('Q8', 'Q9'),  # If works in Prague, work neighborhood
        ('Q10', 'Q11'),  # If studies in Prague, study neighborhood
    ]
    for condition_q, dependent_q in conditional_pairs:
        if condition_q in df.columns and dependent_q in df.columns:
            mask = (df[condition_q] == 'No') & df[dependent_q].notna()
            df.loc[mask, 'data_inconsistency'] = True

    # Step 6: Encode multiple-choice questions
    multiple_choice_columns = ['Q4', 'Q14', 'Q17', 'Q20', 'Q21', 'Q22', 'Q28', 'Q29', 'Q30', 'Q31']
    for col in multiple_choice_columns:
        if col in df.columns:
            dummies = df[col].str.get_dummies(sep=',')
            dummies.columns = [f"{col}_{c}" for c in dummies.columns]
            df = pd.concat([df, dummies], axis=1)

    # Step 7: Age validation
    if 'Q3' in df.columns:
        df['Q3'] = pd.Categorical(df['Q3'], categories=['Under 18', '18 - 24', '25 - 34', '35 - 44', '45 - 54', '55 - 64', '65  or older'], ordered=True)

    # Step 8: Calculate survey duration
    if 'StartDate' in df.columns and 'EndDate' in df.columns:
        df['calculated_duration'] = (df['EndDate'] - df['StartDate']).dt.total_seconds()

    # Step 9: Flag potential data quality issues
    if 'Progress' in df.columns:
        # Convert 'Progress' to numeric, replacing any non-numeric values with NaN
        df['Progress'] = pd.to_numeric(df['Progress'], errors='coerce')
        
        df['potential_issue'] = np.where(
            (df['Progress'] < 100) |  # Incomplete surveys
            (df.get('calculated_duration', pd.Series()) < 60) |  # Very quick responses
            (df.get('data_inconsistency', pd.Series()) == True),  # Inconsistent conditional responses
            True, False
        )
    else:
        print("Warning: 'Progress' column not found in the dataset.")

    # Step 10: Handle language differences
    if 'UserLanguage' in df.columns:
        df['survey_language'] = np.where(df['UserLanguage'] == 'CS', 'CS', 'EN')
    else:
        print("Warning: 'UserLanguage' column not found in the dataset.")

    # Step 11: Process Likert scale questions
    likert_questions = ['Q24', 'Q29', 'Q26', 'Q27']
    for q in likert_questions:
        if q in df.columns:
            df[q] = pd.Categorical(df[q], categories=['Strongly agree', 'Agree', 'Disagree', 'Strongly disagree'], ordered=True)

    # Step 12: Process slider question
    if 'Q24' in df.columns:
        df['Q24'] = pd.to_numeric(df['Q24'], errors='coerce')

    # Debugging: Print column names and data types after processing
    print("\nColumn names after processing:", df.columns.tolist())
    print("\nData types after processing:\n", df.dtypes)

    return df

Seperating Online and Offline Surveys : 

Approach : All Offline surveys were entered into Qualtrix. This was done after the end of the field trip, between 8th and 9th October. Qualtrix records the IP address, location, and time of entry. 

By selecting all IP addresses which entered more than 1 survey, in the netherlands we can isolate all the surveys we entered into the system manually. 

Surveys filled in Prague have to be online since we did not manually input survey forms in prague. 


We categorize the survey data into 4 buckets, This is because of two reasons :
1. Online surveys have conditional logic, which means questions were shown based on previous answers. Ex the question on "What encourages you to bike?" was shown only to people who had selected Yes to Owning a bike or If they used bike sharing.  Offline surveys on the other hand did not have this rich conditional logic as they were printed on paper. 

2. While entering offline surveys back into qualtrics, we initially left the conditional logic on. This meant that certain answers were ignored from the print surveys because Qualtrix did not display the question. After survey 508, it was decided to disable conditional logic on qualtrix to fully represent offline surveys. This is the reason for the 508 and rest differentiation. 


In [2]:
def process_ip_locations(df, world_shapefile_path):
    # Check if required columns exist
    required_columns = ['IPAddress', 'LocationLatitude', 'LocationLongitude']
    if not all(col in df.columns for col in required_columns):
        print("Error: One or more required columns not found in the dataset.")
        return df
    
    # Read world shapefile
    world = gpd.read_file(world_shapefile_path)
    
    # Try to identify the country column
    country_column = next((col for col in ['COUNTRY', 'NAME', 'ADMIN', 'SOVEREIGNT'] if col in world.columns), None)
    if country_column is None:
        print("Error: Could not identify a suitable country column in the world shapefile.")
        return df
    
    print(f"Using '{country_column}' as the country column.")
    
    # Convert longitude and latitude to numeric, replacing any non-numeric values with NaN
    df['LocationLongitude'] = pd.to_numeric(df['LocationLongitude'], errors='coerce')
    df['LocationLatitude'] = pd.to_numeric(df['LocationLatitude'], errors='coerce')
    
    # Create geometry column, filtering out rows with NaN coordinates
    valid_coords = df[['LocationLongitude', 'LocationLatitude']].notna().all(axis=1)
    geometry = [Point(xy) for xy in zip(df.loc[valid_coords, 'LocationLongitude'], df.loc[valid_coords, 'LocationLatitude'])]
    gdf = gpd.GeoDataFrame(df[valid_coords], geometry=geometry, crs="EPSG:4326")
    
    # Perform spatial join
    gdf_with_country = gpd.sjoin(gdf, world[['geometry', country_column]], how="left", predicate="within")
    
    # Categorize as Offline (Netherlands) or Online (rest)
    def categorize_survey_type(country, ip):
        if pd.isna(country) or pd.isna(ip) or ip.lower() in ['anon', 'anonymous', 'na', 'n/a', '']:
            return "Unknown"
        elif country == "Netherlands":
            return "Offline"
        else:
            return "Online"
    
    gdf_with_country['survey_type'] = gdf_with_country.apply(lambda row: categorize_survey_type(row[country_column], row['IPAddress']), axis=1)
    
    # Merge the results back to the original dataframe
    df = df.merge(gdf_with_country[['survey_type']], left_index=True, right_index=True, how='left')
    df['survey_type'] = df['survey_type'].fillna('Unknown')
    
    # Categorize based on survey number
    df['survey_version'] = np.where(df.index <= 508, '508', 'Rest')
    
    return df
processed_df = preprocess_data(df)

# Split and save the data into four categories
categories = {
    'Online_508': (processed_df['survey_type'] == 'Online') & (processed_df['survey_version'] == '508'),
    'Offline_508': (processed_df['survey_type'] == 'Offline') & (processed_df['survey_version'] == '508'),
    'Online_Rest': (processed_df['survey_type'] == 'Online') & (processed_df['survey_version'] == 'Rest'),
    'Offline_Rest': (processed_df['survey_type'] == 'Offline') & (processed_df['survey_version'] == 'Rest')
}

for category, mask in categories.items():
    output_path = f"V:\\Prague_Biking\\Data\\Survey Data\\Processed_Cycling_in_Prague_Survey_{category}.xlsx"
    processed_df[mask].to_excel(output_path, index=False)
    print(f"Processed data for {category} saved to {output_path}")

# Save the full processed data
full_output_path = r"V:\Prague_Biking\Data\Survey Data\Processed_Cycling_in_Prague_Survey_Full.xlsx"
processed_df.to_excel(full_output_path, index=False)
print(f"Full processed data saved to {full_output_path}")

NameError: name 'df' is not defined